# The time locked contract
#### 06.2 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Write a Hash Time Locked Contract
* Use secret (hashed) passwords
* Use time delays
* Limits of passwords on the blockchain

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the 5 accounts and the Purestake credentials

In [ ]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [ ]:
from pyteal import *

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
algod_client.status()["last-round"]

In [ ]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

#### Quick check of asset holdings, otherwise go to ...
- https://bank.testnet.algorand.network
- https://testnet.algoexplorer.io/dispenser

In [ ]:
round(algod_client.account_info(Alice['public'])['amount']/1e6,2)

In [ ]:
print('http://testnet.algoexplorer.io/tx/'+txid)

## The Timelock contract

* Money can only be accessed after a certain amount of time
* *Examples:* vesting periods, deposits

##### Step 1: Fomulate PyTeal conditions

In [ ]:
import random
a = Int( random.randrange(2**32-1) )
random_cond = ( a == a )

# time condition
start_round = algod_client.status()["last-round"] 
time_cond = (
    Txn.first_valid() > Int(start_round+10)           # Earliest payout after 10 rounds from "now"
)

fee_cond = ( Txn.fee() <= Int(1000) )

safety_cond = And(
    Global.group_size() == Int(1),                   # Not part of a TX group
    Txn.type_enum() == TxnType.Payment,
    Txn.rekey_to() == Global.zero_address(),
    Txn.close_remainder_to() == Global.zero_address()
    )
    
timelock_pyteal = And(
    time_cond, 
    random_cond, 
    fee_cond, 
    safety_cond
    )

##### Step 2-3: Compile

In [ ]:
# Step 2: Compile PyTeal -> Teal
timelock_teal = compileTeal(timelock_pyteal, Mode.Signature, version=8)
print(timelock_teal)

# Step 3: Teal -> Bytecode for AVM
Timelock = algod_client.compile(timelock_teal)
print("Smart signature addr: ", Timelock['hash'])
print("Smart signature code: ", Timelock['result'])

##### Step 4: Alice funds and deploys the Smart Signature

In [ ]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(2.5*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Timelock['hash'], amt=amt)

# Step 2+3: sign and sen
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

##### Step 5: Bob asks the smart signature to authorize a transaction
* Be quick! 
* The transaction will fail first
* But it will work after 10 rounds (approx 30-40 seconds)

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(0.1*1e6)

txn = PaymentTxn(sender=Timelock['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Timelock['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)